# GoogLeNet (Inception V1)

In this notebook, I have implemented GoogLeNet (Inception V1) on dataset CIFAR-10 and CIFAR-100.
Auxiliary classification used (exact model).

In [1]:
#!pip install keras
import keras
from keras.datasets import cifar10, cifar100
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Add, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from keras.initializers import glorot_uniform
from keras.regularizers import l2
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras import optimizers

Using TensorFlow backend.


In [2]:
def inception_module(X, f1, f2_in, f2_out, f3_in, f3_out, f4_out, stage, block):
    
    name_sb = str(stage) + str(block)
    
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu', name = 'conv_tower1_' + name_sb)(X)
    # 3x3 conv
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu', name = 'conv_tower2_1_' + name_sb)(X)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu', name = 'conv_tower2_2' + name_sb)(conv3)
    # 5x5 conv
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu', name = 'conv_tower3_1_' + name_sb)(X)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu', name = 'conv_tower3_2_' + name_sb)(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same', name = 'mpool_tower4_' + name_sb)(X)
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu', name = 'conv_tower4_' + name_sb)(pool)
    
    # concatenate filters
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    
    return layer_out

In [3]:
def auxiliary(x, classes, name=None):
    
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = Conv2D(filters=128, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=classes, activation='softmax', name=name)(layer)
    
    return layer

In [21]:
def googlenet(input_shape, classes):
    
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(32, (3, 3), strides = (2, 2), name = 'conv_pre_inception_1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_pre_inception_1')(X)
    
    # Stage 2
    X = Conv2D(192, (3, 3), strides = (1, 1), name = 'conv_pre_inception_2', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_2')(X)
    X = Activation('relu')(X)
    X = Conv2D(192, (3, 3), strides = (1, 1), name = 'conv_pre_inception_3', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_pre_inception_2')(X)
    
    # Stage 3
    X = inception_module(X, 64, 96, 128, 16, 32, 32, 3, 'a')
    X = inception_module(X, 128, 128, 192, 32, 96, 64, 3, 'b')
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_inception_3')(X)
    
    # Stage 4
    X = inception_module(X, 192, 96, 208, 16, 48, 64, 4, 'a')
    aux1  = auxiliary(X, classes, name='aux1')
    X = inception_module(X, 160, 112, 224, 24, 64, 64, 4, 'b')
    X = inception_module(X, 128, 128, 256, 24, 24, 64, 4, 'c')
    X = inception_module(X, 112, 144, 288, 32, 64, 64, 4, 'd')
    aux2  = auxiliary(X, classes, name='aux2')
    X = inception_module(X, 256, 120, 320, 32, 128, 128, 4, 'e')
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_inception_4')(X)
    
    # Stage 5
    X = inception_module(X, 256, 160, 320, 32, 128, 128, 5, 'a')
    X = inception_module(X, 384, 192, 384, 48, 128, 128, 5, 'b')
    X = AveragePooling2D((7, 7), strides=(1, 1), name = 'apool_inception_5')(X)
    
    # Stage 6
    X = Dropout(0.4)(X)
    X = Flatten()(X)
    X = Dense(1000, activation='relu', kernel_initializer = glorot_uniform(seed=0))(X)
    main = Dense(classes, activation='softmax', name='main', kernel_initializer = glorot_uniform(seed=0))(X)
    
    aux1 = keras.layers.Lambda(lambda x: x * 0.3)(aux1)                                         # Method 1
    aux2 = keras.layers.Lambda(lambda x: x * 0.3)(aux2)                                         # Method 1
    totalloss = keras.layers.Add()([aux1, aux2, main])                                          # Method 1
    inceptionv1 = Model(inputs = X_input, outputs = [totalloss], name='InceptionV1')            # Method 1
    #inceptionv1 = Model(inputs = X_input, outputs = [main, aux1, aux2], name='InceptionV1')    # Method 2

    return inceptionv1

## CIFAR-10

In [ ]:
# Load data
(c10_x_train, c10_y_train), (c10_x_test, c10_y_test) = cifar10.load_data()

# Hyper-parameters
epochs = 20
batch_size = 128
num_classes = 10
input_shape = c10_x_train.shape[1:]

# Pre-processing
c10_x_train = c10_x_train.astype('float32')
c10_x_test = c10_x_test.astype('float32')
c10_x_train = c10_x_train/255
c10_x_test = c10_x_test/255
c10_y_train = keras.utils.to_categorical(c10_y_train, num_classes)
c10_y_test = keras.utils.to_categorical(c10_y_test, num_classes)

In [8]:
googlenet = googlenet(input_shape, num_classes)
googlenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
googlenet.summary()

Model: "InceptionV1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv_pre_inception_1 (Conv2D)   (None, 18, 18, 32)   896         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_pre_inception_1 (BatchNormal (None, 18, 18, 32)   128         conv_pre_inception_1[0][0]       
________________________________________________________________________________________

In [10]:
googlenet.fit(c10_x_train, c10_y_train, epochs = 5, batch_size = 128)

Epoch 1/5
50000/50000 [==============================] - 94s 2ms/step - loss: 1.5870 - accuracy: 0.4073
Epoch 2/5
50000/50000 [==============================] - 80s 2ms/step - loss: 1.1537 - accuracy: 0.5845
Epoch 3/5
50000/50000 [==============================] - 79s 2ms/step - loss: 0.9391 - accuracy: 0.6674
Epoch 4/5
50000/50000 [==============================] - 80s 2ms/step - loss: 0.8034 - accuracy: 0.7172
Epoch 5/5
50000/50000 [==============================] - 80s 2ms/step - loss: 0.6925 - accuracy: 0.7590


In [12]:
scores = googlenet.evaluate(c10_x_test, c10_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 8s 775us/step
Test loss: 0.8266890961647033
Test accuracy: 0.7185999751091003


In [13]:
googlenet.fit(c10_x_train, c10_y_train, epochs = 10, batch_size = 128) # Method 2

Epoch 1/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.6179 - accuracy: 0.7874
Epoch 2/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.5523 - accuracy: 0.8084
Epoch 3/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.4892 - accuracy: 0.8286
Epoch 4/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.4332 - accuracy: 0.8499
Epoch 5/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.3976 - accuracy: 0.8599
Epoch 6/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.3588 - accuracy: 0.8751
Epoch 7/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.3229 - accuracy: 0.8870
Epoch 8/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.2807 - accuracy: 0.9018
Epoch 9/10
50000/50000 [==============================] - 80s 2ms/step - loss: 0.2671 - accuracy: 0.9063
Epoch 10/10
50000/50000 [==============================

In [14]:
scores = googlenet.evaluate(c10_x_test, c10_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 726us/step
Test loss: 0.7769441133975983
Test accuracy: 0.7865999937057495


In [15]:
googlenet.fit(c10_x_train, c10_y_train, epochs = 5, batch_size = 128)

Epoch 1/5
50000/50000 [==============================] - 80s 2ms/step - loss: 0.2256 - accuracy: 0.9201
Epoch 2/5
50000/50000 [==============================] - 80s 2ms/step - loss: 0.2094 - accuracy: 0.9267
Epoch 3/5
50000/50000 [==============================] - 80s 2ms/step - loss: 0.1870 - accuracy: 0.9337
Epoch 4/5
50000/50000 [==============================] - 80s 2ms/step - loss: 0.1876 - accuracy: 0.9329
Epoch 5/5
50000/50000 [==============================] - 80s 2ms/step - loss: 0.1561 - accuracy: 0.9449


In [16]:
scores = googlenet.evaluate(c10_x_test, c10_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 720us/step
Test loss: 0.9074650887966156
Test accuracy: 0.7718999981880188


In [9]:
googlenet.fit(c10_x_train, [c10_y_train, c10_y_train, c10_y_train], epochs = 25, batch_size = 128)  # Method 1

Epoch 1/25
50000/50000 [==============================] - 96s 2ms/step - loss: 4.7755 - main_loss: 1.6491 - aux1_loss: 1.5289 - aux2_loss: 1.5958 - main_accuracy: 0.3742 - aux1_accuracy: 0.4290 - aux2_accuracy: 0.3979
Epoch 2/25
50000/50000 [==============================] - 82s 2ms/step - loss: 3.4071 - main_loss: 1.1734 - aux1_loss: 1.0927 - aux2_loss: 1.1404 - main_accuracy: 0.5768 - aux1_accuracy: 0.6077 - aux2_accuracy: 0.5896
Epoch 3/25
50000/50000 [==============================] - 82s 2ms/step - loss: 2.7062 - main_loss: 0.9280 - aux1_loss: 0.8675 - aux2_loss: 0.9103 - main_accuracy: 0.6721 - aux1_accuracy: 0.6942 - aux2_accuracy: 0.6809: 8s - loss: 2.7250 - main_loss: 0.9349 - aux1_loss: 0.8740 - aux2_loss: 0.9162 - main_accuracy: 0.6684 - aux1_accuracy: 0.690 - ETA: 3s - loss: 2.7182 - main_loss: 0.9326 - aux1_loss: 0.8716 - aux2_loss: 0.9139 - main_accuracy: 0.6701 - aux1_accuracy: 0.6926 - aux
Epoch 4/25
50000/50000 [==============================] - 82s 2ms/step - loss: 2.

In [10]:
scores = googlenet.evaluate(c10_x_test, [c10_y_test, c10_y_test, c10_y_test])
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 9s 875us/step
Test loss: 3.162414232349396
Test accuracy: 1.0123772621154785


In [11]:
googlenet.save('googlenet_c10.h5')
googlenet.save_weights('googlenet_c10_weights.h5')

# CIFAR - 100

In [15]:
# Load data
(c100_x_train, c100_y_train), (c100_x_test, c100_y_test) = cifar100.load_data()

# Hyper-parameters
c100_epochs = 25
c100_batch_size = 128
c100_num_classes = 100
c100_input_shape = c100_x_train.shape[1:]

# Pre-processing
c100_x_train = c100_x_train.astype('float32')
c100_x_test = c100_x_test.astype('float32')
c100_x_train = c100_x_train/255
c100_x_test = c100_x_test/255
c100_y_train = keras.utils.to_categorical(c100_y_train, c100_num_classes)
c100_y_test = keras.utils.to_categorical(c100_y_test, c100_num_classes)

In [16]:
c100_googlenet = googlenet(input_shape = c100_input_shape, classes = c100_num_classes)
c100_googlenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#c100_googlenet.summary()

In [9]:
# Method 1
c100_googlenet.fit(c100_x_train, [c100_y_train, c100_y_train, c100_y_train], validation_data=(c100_x_test, [c100_y_test, c100_y_test, c100_y_test]), epochs = c100_epochs, batch_size = c100_batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 82s 2ms/step - loss: 0.9787 - main_loss: 0.2493 - aux1_loss: 0.3613 - aux2_loss: 0.3684 - main_accuracy: 0.9199 - aux1_accuracy: 0.8816 - aux2_accuracy: 0.8828 - val_loss: 12.4432 - val_main_loss: 4.3773 - val_aux1_loss: 4.0579 - val_aux2_loss: 3.9761 - val_main_accuracy: 0.4004 - val_aux1_accuracy: 0.4138 - val_aux2_accuracy: 0.4062n_loss: 0.1937 - aux1_loss: 0.3122 - aux2_loss: 0.3079 - main_accuracy: 0.9373 - aux1_accuracy: 0.8975 - aux2_accura - ETA: 49s - loss: 0.8232 - main_loss: 0.1975 - aux1_loss: 0.3158 - aux2_loss: 0.3099 - main_accuracy: 0.9365 - aux1_accuracy: 0.8966 - a - ETA: 47s - loss: 0.8316 - main_loss: 0.2012 - aux1_loss: 0.3169 - aux2_loss: 0.3135 - main_accuracy: 0.9357 - aux1_accuracy: 0.8959 - aux2_ - ETA: 46s - loss: 0.8490 - main_loss: 0.2074 - aux1_loss: 0.3221 - aux2_loss: 0.3195 - main_accuracy: 0.9339 - aux1_accuracy: 0.8941 - aux2_accu - ETA: 44s - l

In [12]:
scores = c100_googlenet.evaluate(c100_x_test, [c100_y_test, c100_y_test, c100_y_test])
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 6s 640us/step
Test loss: 12.912689134216308
Test accuracy: 4.190948486328125


In [13]:
c100_googlenet.save('googlenet_c100.h5')
c100_googlenet.save_weights('googlenet_c100_weights.h5')

In [17]:
c100_googlenet.fit(c100_x_train, c100_y_train, epochs = c100_epochs, batch_size = c100_batch_size) # Method 2

Epoch 1/25
50000/50000 [==============================] - 83s 2ms/step - loss: 4.0550 - accuracy: 0.0753
Epoch 2/25
50000/50000 [==============================] - 77s 2ms/step - loss: 3.5255 - accuracy: 0.1527
Epoch 3/25
50000/50000 [==============================] - 77s 2ms/step - loss: 3.1837 - accuracy: 0.2131
Epoch 4/25
50000/50000 [==============================] - 78s 2ms/step - loss: 2.8986 - accuracy: 0.2655
Epoch 5/25
50000/50000 [==============================] - 77s 2ms/step - loss: 2.6458 - accuracy: 0.3131
Epoch 6/25
50000/50000 [==============================] - 78s 2ms/step - loss: 2.4445 - accuracy: 0.3518
Epoch 7/25
50000/50000 [==============================] - 78s 2ms/step - loss: 2.2635 - accuracy: 0.3918
Epoch 8/25
50000/50000 [==============================] - 78s 2ms/step - loss: 2.1202 - accuracy: 0.4225
Epoch 9/25
50000/50000 [==============================] - 78s 2ms/step - loss: 1.9820 - accuracy: 0.4530
Epoch 10/25
50000/50000 [==============================

In [20]:
scores = c100_googlenet.evaluate(c100_x_test, c100_y_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 6s 621us/step
Test loss: 2.4383833072662355
Test accuracy: 0.454800009727478
